# Big Data Project: House Prices - Advanced Regression Techniques
Presentazione del progetto di esame dell'a.a. 2021-2022 di Big Data del CLM in Ingegneria Informatica. Il lavoro presentato comprende lo svolgimento della competizione Kaggle __[House Prices - Avanced Regression Techniques]([http://url](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques))__, in cui viene richiesta un'analisi predittiva sul valore di mercato di abitazioni, sulla base di variabili descrittive di una moltitudine di caratteristiche degli immobili che, in diversa misura, ne influiscono sul prezzo.

# ![banner](./imgs/housesbanner.png)

Il __[dataset](http://jse.amstat.org/v19n3/decock.pdf)__ a disposizione è un file `csv` contenente informazioni sulle abitazioni della cittadina di __[Ames](https://www.google.com/maps/place/Ames,+IA,+USA/@42.0258192,-93.6964163,12z/data=!3m1!4b1!4m5!3m4!1s0x87ee70624634a06b:0x273156083cc75200!8m2!3d42.0307812!4d-93.6319131)__, nello stato americano dell'Iowa. In compendio al dataset, viene anche fornita una dettagliata __[descrizione](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data)__ dei singoli campi:

- **SalePrice** - the property's sale price in dollars. This is the target variable that you're trying to predict.
- MSSubClass: The building class
- MSZoning: The general zoning classification
- LotFrontage: Linear feet of street connected to property
- LotArea: Lot size in square feet
- Street: Type of road access
- Alley: Type of alley access
- LotShape: General shape of property
- LandContour: Flatness of the property
- Utilities: Type of utilities available
- LotConfig: Lot configuration
- LandSlope: Slope of property
- Neighborhood: Physical locations within Ames city limits
- Condition1: Proximity to main road or railroad
- Condition2: Proximity to main road or railroad (if a second is present)
- BldgType: Type of dwelling
- HouseStyle: Style of dwelling
- OverallQual: Overall material and finish quality
- OverallCond: Overall condition rating
- YearBuilt: Original construction date
- YearRemodAdd: Remodel date
- RoofStyle: Type of roof
- RoofMatl: Roof material
- Exterior1st: Exterior covering on house
- Exterior2nd: Exterior covering on house (if more than one material)
- MasVnrType: Masonry veneer type
- MasVnrArea: Masonry veneer area in square feet
- ExterQual: Exterior material quality
- ExterCond: Present condition of the material on the exterior
- Foundation: Type of foundation
- BsmtQual: Height of the basement
- BsmtCond: General condition of the basement
- BsmtExposure: Walkout or garden level basement walls
- BsmtFinType1: Quality of basement finished area
- BsmtFinSF1: Type 1 finished square feet
- BsmtFinType2: Quality of second finished area (if present)
- BsmtFinSF2: Type 2 finished square feet
- BsmtUnfSF: Unfinished square feet of basement area
- TotalBsmtSF: Total square feet of basement area
- Heating: Type of heating
- HeatingQC: Heating quality and condition
- CentralAir: Central air conditioning
- Electrical: Electrical system
- 1stFlrSF: First Floor square feet
- 2ndFlrSF: Second floor square feet
- LowQualFinSF: Low quality finished square feet (all floors)
- GrLivArea: Above grade (ground) living area square feet
- BsmtFullBath: Basement full bathrooms
- BsmtHalfBath: Basement half bathrooms
- FullBath: Full bathrooms above grade
- HalfBath: Half baths above grade
- Bedroom: Number of bedrooms above basement level
- Kitchen: Number of kitchens
- KitchenQual: Kitchen quality
- TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
- Functional: Home functionality rating
- Fireplaces: Number of fireplaces
- FireplaceQu: Fireplace quality
- GarageType: Garage location
- GarageYrBlt: Year garage was built
- GarageFinish: Interior finish of the garage
- GarageCars: Size of garage in car capacity
- GarageArea: Size of garage in square feet
- GarageQual: Garage quality
- GarageCond: Garage condition
- PavedDrive: Paved driveway
- WoodDeckSF: Wood deck area in square feet
- OpenPorchSF: Open porch area in square feet
- EnclosedPorch: Enclosed porch area in square feet
- 3SsnPorch: Three season porch area in square feet
- ScreenPorch: Screen porch area in square feet
- PoolArea: Pool area in square feet
- PoolQC: Pool quality
- Fence: Fence quality
- MiscFeature: Miscellaneous feature not covered in other categories
- MiscVal: $Value of miscellaneous feature
- MoSold: Month Sold
- YrSold: Year Sold
- SaleType: Type of sale
- SaleCondition: Condition of sale

Kaggle mette a disposizione i file `train.csv` e `test.csv`, necessari rispettivamente per l'addestramento del modello predittivo e la valutazione finale (su cui verrà calcolato lo score della competizione), che presentano la stessa struttura eccetto naturalmente per la colonna **SalePrice**, assente in `test.csv`, associata alla variabile target su cui bisognerà effettuare la predizione. 

Quello presentato è un problema di regressione, in cui risulta cruciale anche lo stage di analisi e preparazione dei dati da fornire al modello di apprendimento, in quanto il numero di campi per ogni record/casa è considerevole (79) e la mancata imputazione di dati mancanti e/o errati, così come la scelta di feature non rilevanti ai fini della valutazione del costo, può significativamente impattare l'efficacia dei regressori utilizzati.


## 0. Ambiente di lavoro e tecnologie utilizzate
L'analisi e l'addestramento sono interamente realizzati in Python (v3.8.2) tramite le rispettive API per l'interfacciamento con le tecnologie di Big Data per storage e processing distribuiti. 

In particolare, il framework per l'implementazione dell'architettura Big Data è composto da:
- ``Hadoop Distributed File System`` (HDFS v3.2.1) per la gestione (distribuita) delle repository contenenti i file di input/output ed elaborazione intermedia. **Nota**: il cluster Hadoop, in questo caso, è costituito da un solo nodo, in quanto la dimensione del dataset di lavoro è di poche centinaia di kB (460.68 kB per `train.csv` e 451.41 kB per `test.csv`), che rientra in un solo blocco HDFS, la cui dimensione standard è di 128 MB.
- ``Apache Spark`` (v3.3.0) come engine di data processing distribuito e la relativa API Python `pyspark`, che contiene sia le funzionalità di analisi dei dati secondo la filosofia dei DataFrame (ed RDD) (come nel caso di `pyspark.sql`), sia gli __[algoritmi di machine learning più diffusi](https://spark.apache.org/docs/latest/ml-guide.html)__ (localizzati in `pyspark.ml`).

Analisi ed elaborazioni sul dataset sono in gran parte svolte tramite le funzionalità messe a disposizione da `pyspark`, che gestisce in maniera autonoma l'esecuzione distribuita su cluster e il monitoraggio delle risorse e degli stati di completamento dei job lanciati. In alcuni casi verranno anche sfruttate le funzionalità aggiuntive dei DataFrame `pandas`, che, seppur di natura simile a quella dei DF Spark, presentano una maggior compatibilità con alcuni strumenti di analisi secondari come quelli di visualizzazione (`matplotlib` e `seaborn`). Ciononostante, l'utilizzo di DF `pandas` viene mantenuto fortemente limitato, in quanto il metodo `toPandas()`, che effettua la conversione di DataFrame a partire da Spark, __[sposta lo storage e l'elaborazione in locale](https://stackoverflow.com/questions/29226210/what-is-the-spark-dataframe-method-topandas-actually-doing)__, annullando di fatto la distribuzione e parallelizzazione delle computazioni. In uno scenario reale di Big Data naturalmente ciò è fortemente indesiderato in quanto una singola macchina potrebbe non disporre delle risorse necessarie per gestire internamente un quantitativo di dati che era precedentemente sparso su diversi nodi del cluster. Pertanto, ai fini di rimanere il più fedeli possibile ad una reale, pratica analisi Big Data, `toPandas()` verrà applicato in poche occasioni e solo su sottoinsiemi (e.g., selezioni, proiezioni, aggregazioni, ecc...) del dataset completo.

Il progetto verrà descritto in dettaglio nel presente notebook Jupyter, utilizzando il markdown per argomentare le scelte effettuate e commentare i risultati ottenuti, e lasciando ai commenti entro le celle di codice Python per il chiarimento degli aspetti implementativi.



